# Analyse Tokeniser 

In [11]:
model_dir = "./tests/temp/biobert"
bucket = "aegovan-data"
trainfile = "s3://{}/processed_dataset/train_multiclass.json".format(bucket)

column="normalised_abstract"

In [2]:
data_file="input_data.json"


In [3]:
from pytorch_pretrained_bert import BertTokenizer


tokeniser = BertTokenizer.from_pretrained(model_dir, do_lower_case=False)

In [4]:
import boto3

def download_single_file(bucket_name_path, local_path):
    index = bucket_name_path.find("://")

    # remove the s3:// if present
    if index > -1:
        bucket_name_path = bucket_name_path[index + 3:]

    key_start_index = bucket_name_path.find("/")
    bucket_name = bucket_name_path
    key = "/"
    if key_start_index > -1:
        bucket_name = bucket_name_path[0:key_start_index]
        key = bucket_name_path[key_start_index + 1:]
        
    client = boto3.resource('s3')
    client.Bucket(bucket_name).download_file(key, local_path)



In [70]:
import json
def get_counts(input_file, column):
    with open(input_file, "r") as f:
        data = json.load(f)
            
    counts = []
    for _, line in data[column].items():
        tokens = tokeniser.tokenize(line)
        counts.append({
            "tokens" : tokens,
            "token_len" : len(tokens)
        })
    
    return counts

In [ ]:
download_single_file(trainfile, data_file) 

In [71]:
counts = get_counts(data_file, column)

In [72]:
import pandas as pd

df = pd.DataFrame( counts)

In [73]:
df.sample(n=10)

,token_len,tokens
2425,512,"[St, ##im, ##ulation, of, mature, T, cells, ac..."
142,513,"[D, -, sites, are, a, class, of, MA, ##P, ##K,..."
560,328,"[Mi, ##to, ##gen, -, activated, protein, kinas..."
1656,469,"[Trans, ##cription, in, e, ##uka, ##ryo, ##tic..."
1121,267,"[Cy, ##c, ##lin, -, dependent, kinase, ##s, (,..."
2366,577,"[P, ##60, ##5, ##6, ##8, stimulation, of, T, l..."
644,357,"[We, have, recently, shown, that, P, ##18, ##0..."
134,513,"[D, -, sites, are, a, class, of, MA, ##P, ##K,..."
1353,475,"[Variations, in, in, ##tra, ##cellular, levels..."
3111,432,"[Two, genes, on, chromosome, 21, ,, namely, Q,..."


In [79]:
df.describe()

,token_len
count,3223.000000
mean,375.686007
std,102.796268
min,57.000000
25%,294.000000
50%,368.000000
75%,451.000000
max,795.000000


In [77]:
df.shape,df.query("token_len > 510").shape

((3223, 2), (367, 2))